In [1]:
import pandas as pd
import numpy as np
from scipy import sparse
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
from sklearn.metrics import mean_squared_error

In [2]:
train_joke_df = pd.read_csv(r'..\data\recsys-in-practice\train_joke_df.csv')
test_joke_df_nofactrating = pd.read_csv(r'..\data\recsys-in-practice\test_joke_df_nofactrating.csv', index_col=0)

In [3]:
train_joke_df["UID"] = train_joke_df["UID"].astype(int)
train_joke_df["JID"] = train_joke_df["JID"].astype(int)

In [4]:
train_joke_df["UID"] = train_joke_df["UID"] - 1
train_joke_df["JID"] = train_joke_df["JID"] - 1

In [5]:
npusers_arr = np.zeros((24983, 100), dtype=np.half)

for row in tqdm(train_joke_df.values):
    npusers_arr[int(row[0]), int(row[1])] = row[2]

  0%|          | 0/1448364 [00:00<?, ?it/s]

In [6]:
users_df = pd.DataFrame(npusers_arr, index=range(0, npusers_arr.shape[0]), columns=[f'j_{i}' for i in range(100)])
users_df

,j_0,j_1,j_2,j_3,j_4,j_5,j_6,j_7,j_8,j_9,...,j_90,j_91,j_92,j_93,j_94,j_95,j_96,j_97,j_98,j_99
0,-7.820312,8.789062,-9.656250,-8.156250,-7.519531,-8.500000,-9.851562,4.171875,-8.976562,-4.761719,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-5.628906,0.000000,0.000000,0.000000
1,4.078125,-0.290039,6.359375,4.371094,0.000000,-9.656250,-0.729980,-5.339844,8.882812,9.218750,...,2.820312,-4.949219,-0.290039,7.859375,-0.189941,0.000000,3.060547,0.000000,-4.320312,0.000000
2,0.000000,0.000000,0.000000,0.000000,9.031250,9.273438,9.031250,9.273438,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,8.351562,0.000000,0.000000,1.799805,0.000000,-2.820312,6.210938,0.000000,1.839844,...,0.000000,0.000000,0.000000,0.529785,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,8.500000,4.609375,-4.171875,-5.390625,0.000000,0.000000,7.039062,0.000000,-0.439941,0.000000,...,0.000000,5.578125,4.269531,5.191406,5.730469,1.549805,0.000000,6.550781,1.799805,1.599609
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24978,0.439941,0.000000,0.000000,2.330078,0.000000,6.750000,-8.789062,-0.529785,0.000000,0.000000,...,8.828125,-1.209961,9.218750,-6.699219,0.000000,9.031250,6.550781,8.687500,0.000000,7.429688
24979,0.000000,-8.156250,8.593750,9.078125,0.870117,-8.929688,-3.500000,5.781250,-8.109375,0.000000,...,-1.169922,-5.730469,0.000000,0.239990,9.218750,-8.203125,0.000000,-8.593750,9.132812,8.453125
24980,0.000000,0.000000,0.000000,0.000000,-7.769531,0.000000,0.000000,-6.750000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
24981,0.000000,0.000000,0.000000,0.000000,-9.710938,0.000000,4.558594,-8.296875,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [7]:
train_joke_df = train_joke_df.merge(users_df, how='inner', left_on='UID',  right_index=True)
train_joke_df

,UID,JID,Rating,j_0,j_1,j_2,j_3,j_4,j_5,j_6,...,j_90,j_91,j_92,j_93,j_94,j_95,j_96,j_97,j_98,j_99
0,18028,5,-1.26,0.0,0.0,0.0,0.0,1.120117,-1.259766,3.300781,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
18990,18028,39,8.30,0.0,0.0,0.0,0.0,1.120117,-1.259766,3.300781,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
50486,18028,6,3.30,0.0,0.0,0.0,0.0,1.120117,-1.259766,3.300781,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
69643,18028,15,-9.27,0.0,0.0,0.0,0.0,1.120117,-1.259766,3.300781,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
124589,18028,14,8.11,0.0,0.0,0.0,0.0,1.120117,-1.259766,3.300781,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1303561,21378,61,0.15,0.0,0.0,0.0,0.0,-2.820312,0.000000,-8.007812,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.189941,0.0,0.0
1331118,21378,45,1.31,0.0,0.0,0.0,0.0,-2.820312,0.000000,-8.007812,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.189941,0.0,0.0
1368576,21378,48,-0.68,0.0,0.0,0.0,0.0,-2.820312,0.000000,-8.007812,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.189941,0.0,0.0
1376073,21378,53,2.33,0.0,0.0,0.0,0.0,-2.820312,0.000000,-8.007812,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.189941,0.0,0.0


In [10]:
for i, row in enumerate(tqdm(train_joke_df.values)):
    j = int(row[1])
    #print(train_joke_df.iloc[i, 3 + j])
    assert np.isclose(row[2], train_joke_df.iloc[i, 3 + j], atol=1e-2)
    train_joke_df.iloc[i, 3 + j] = 0
    #print(train_joke_df.iloc[i, 3 + j])


0it [00:00, ?it/s]

In [11]:
for i, row in enumerate(tqdm(train_joke_df.values)):
    j = int(row[1])
    #print(train_joke_df.iloc[i, 3 + j])
    #print(row[3 + joke_id])
    assert np.isclose(0, train_joke_df.iloc[i, 3 + j])


  0%|          | 0/1448364 [00:00<?, ?it/s]

In [12]:
train_joke_df

,UID,JID,Rating,j_0,j_1,j_2,j_3,j_4,j_5,j_6,...,j_90,j_91,j_92,j_93,j_94,j_95,j_96,j_97,j_98,j_99
0,18028,5,-1.26,0.0,0.0,0.0,0.0,1.120117,0.000000,3.300781,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
18990,18028,39,8.30,0.0,0.0,0.0,0.0,1.120117,-1.259766,3.300781,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
50486,18028,6,3.30,0.0,0.0,0.0,0.0,1.120117,-1.259766,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
69643,18028,15,-9.27,0.0,0.0,0.0,0.0,1.120117,-1.259766,3.300781,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
124589,18028,14,8.11,0.0,0.0,0.0,0.0,1.120117,-1.259766,3.300781,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1303561,21378,61,0.15,0.0,0.0,0.0,0.0,-2.820312,0.000000,-8.007812,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.189941,0.0,0.0
1331118,21378,45,1.31,0.0,0.0,0.0,0.0,-2.820312,0.000000,-8.007812,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.189941,0.0,0.0
1368576,21378,48,-0.68,0.0,0.0,0.0,0.0,-2.820312,0.000000,-8.007812,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.189941,0.0,0.0
1376073,21378,53,2.33,0.0,0.0,0.0,0.0,-2.820312,0.000000,-8.007812,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.189941,0.0,0.0


In [15]:
train_joke_df.tail(30)

,UID,JID,Rating,j_0,j_1,j_2,j_3,j_4,j_5,j_6,...,j_90,j_91,j_92,j_93,j_94,j_95,j_96,j_97,j_98,j_99
460572,21378,28,3.40,0.0,0.0,0.0,0.0,-2.820312,0.0,-8.007812,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.189941,0.0,0.0
485937,21378,33,-4.61,0.0,0.0,0.0,0.0,-2.820312,0.0,-8.007812,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.189941,0.0,0.0
488228,21378,6,-8.01,0.0,0.0,0.0,0.0,-2.820312,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.189941,0.0,0.0
509988,21378,60,-2.62,0.0,0.0,0.0,0.0,-2.820312,0.0,-8.007812,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.189941,0.0,0.0
579667,21378,25,1.50,0.0,0.0,0.0,0.0,-2.820312,0.0,-8.007812,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.189941,0.0,0.0
662520,21378,67,-0.53,0.0,0.0,0.0,0.0,-2.820312,0.0,-8.007812,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.189941,0.0,0.0
702969,21378,34,2.28,0.0,0.0,0.0,0.0,-2.820312,0.0,-8.007812,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.189941,0.0,0.0
703279,21378,41,1.31,0.0,0.0,0.0,0.0,-2.820312,0.0,-8.007812,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.189941,0.0,0.0
739560,21378,15,-7.43,0.0,0.0,0.0,0.0,-2.820312,0.0,-8.007812,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.189941,0.0,0.0
831839,21378,38,-3.35,0.0,0.0,0.0,0.0,-2.820312,0.0,-8.007812,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.189941,0.0,0.0


In [16]:
rows, r_pos = np.unique(train_joke_df.values[:,0], return_inverse=True)
cols, c_pos = np.unique(train_joke_df.values[:,1], return_inverse=True)

In [17]:
rows

array([0.0000e+00, 1.0000e+00, 2.0000e+00, ..., 2.4980e+04, 2.4981e+04,
       2.4982e+04])

In [18]:
cols

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25.,
       26., 27., 28., 29., 30., 31., 32., 33., 34., 35., 36., 37., 38.,
       39., 40., 41., 42., 43., 44., 45., 46., 47., 48., 49., 50., 51.,
       52., 53., 54., 55., 56., 57., 58., 59., 60., 61., 62., 63., 64.,
       65., 66., 67., 68., 69., 70., 71., 72., 73., 74., 75., 76., 77.,
       78., 79., 80., 81., 82., 83., 84., 85., 86., 87., 88., 89., 90.,
       91., 92., 93., 94., 95., 96., 97., 98., 99.])

In [19]:
train_df, valid_df, train_queries, valid_queries = train_test_split(train_joke_df, r_pos, test_size=0.3, random_state=42)

In [42]:
valid_df[valid_df['UID'] == 44]

,UID,JID,Rating,j_0,j_1,j_2,j_3,j_4,j_5,j_6,...,j_90,j_91,j_92,j_93,j_94,j_95,j_96,j_97,j_98,j_99
51504,44,71,3.20,8.789062,0.0,5.488281,0.919922,0.0,0.0,4.511719,...,0.0,3.880859,3.830078,0.0,0.290039,2.769531,1.459961,-2.769531,2.179688,-1.209961
693981,44,22,8.83,8.789062,0.0,5.488281,0.919922,0.0,0.0,4.511719,...,0.0,3.880859,3.830078,0.0,0.290039,2.769531,1.459961,-2.769531,2.179688,-1.209961
1239916,44,76,3.35,8.789062,0.0,5.488281,0.919922,0.0,0.0,4.511719,...,0.0,3.880859,3.830078,0.0,0.290039,2.769531,1.459961,-2.769531,2.179688,-1.209961
826257,44,6,4.51,8.789062,0.0,5.488281,0.919922,0.0,0.0,0.000000,...,0.0,3.880859,3.830078,0.0,0.290039,2.769531,1.459961,-2.769531,2.179688,-1.209961
1089514,44,78,2.77,8.789062,0.0,5.488281,0.919922,0.0,0.0,4.511719,...,0.0,3.880859,3.830078,0.0,0.290039,2.769531,1.459961,-2.769531,2.179688,-1.209961
226234,44,79,2.77,8.789062,0.0,5.488281,0.919922,0.0,0.0,4.511719,...,0.0,3.880859,3.830078,0.0,0.290039,2.769531,1.459961,-2.769531,2.179688,-1.209961
1413678,44,80,2.77,8.789062,0.0,5.488281,0.919922,0.0,0.0,4.511719,...,0.0,3.880859,3.830078,0.0,0.290039,2.769531,1.459961,-2.769531,2.179688,-1.209961
643300,44,92,3.83,8.789062,0.0,5.488281,0.919922,0.0,0.0,4.511719,...,0.0,3.880859,0.000000,0.0,0.290039,2.769531,1.459961,-2.769531,2.179688,-1.209961
992047,44,66,8.45,8.789062,0.0,5.488281,0.919922,0.0,0.0,4.511719,...,0.0,3.880859,3.830078,0.0,0.290039,2.769531,1.459961,-2.769531,2.179688,-1.209961
585603,44,57,0.83,8.789062,0.0,5.488281,0.919922,0.0,0.0,4.511719,...,0.0,3.880859,3.830078,0.0,0.290039,2.769531,1.459961,-2.769531,2.179688,-1.209961


In [ ]:
train_df[train_df['UID'] == 44]

In [61]:
for i, row in enumerate(tqdm(valid_df.values)):
    u = int(row[0])
    j = int(row[1])
    
    train_df.loc[train_df['UID'] == u, f'j_{j}'] = 0

  0%|          | 0/434510 [00:00<?, ?it/s]

504867     0.000000
1214727    5.488281
1421955    5.488281
919959     5.488281
553877     5.488281
             ...   
650346     5.488281
1230384    5.488281
746078     5.488281
258762     5.488281
113054     5.488281
Name: j_2, Length: 69, dtype: float16

504867     0.0
1214727    0.0
1421955    0.0
919959     0.0
553877     0.0
          ... 
650346     0.0
1230384    0.0
746078     0.0
258762     0.0
113054     0.0
Name: j_2, Length: 69, dtype: float16

In [62]:
train_df[train_df['UID'] == 44]

,UID,JID,Rating,j_0,j_1,j_2,j_3,j_4,j_5,j_6,...,j_90,j_91,j_92,j_93,j_94,j_95,j_96,j_97,j_98,j_99
504867,44,2,5.49,8.789062,0.0,0.0,0.919922,0.0,0.0,4.511719,...,0.0,3.880859,3.830078,0.0,0.290039,2.769531,1.459961,-2.769531,2.179688,-1.209961
1214727,44,44,7.52,8.789062,0.0,0.0,0.919922,0.0,0.0,4.511719,...,0.0,3.880859,3.830078,0.0,0.290039,2.769531,1.459961,-2.769531,2.179688,-1.209961
1421955,44,73,1.02,8.789062,0.0,0.0,0.919922,0.0,0.0,4.511719,...,0.0,3.880859,3.830078,0.0,0.290039,2.769531,1.459961,-2.769531,2.179688,-1.209961
919959,44,21,5.58,8.789062,0.0,0.0,0.919922,0.0,0.0,4.511719,...,0.0,3.880859,3.830078,0.0,0.290039,2.769531,1.459961,-2.769531,2.179688,-1.209961
553877,44,20,6.41,8.789062,0.0,0.0,0.919922,0.0,0.0,4.511719,...,0.0,3.880859,3.830078,0.0,0.290039,2.769531,1.459961,-2.769531,2.179688,-1.209961
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
650346,44,91,3.88,8.789062,0.0,0.0,0.919922,0.0,0.0,4.511719,...,0.0,0.000000,3.830078,0.0,0.290039,2.769531,1.459961,-2.769531,2.179688,-1.209961
1230384,44,30,5.39,8.789062,0.0,0.0,0.919922,0.0,0.0,4.511719,...,0.0,3.880859,3.830078,0.0,0.290039,2.769531,1.459961,-2.769531,2.179688,-1.209961
746078,44,50,5.15,8.789062,0.0,0.0,0.919922,0.0,0.0,4.511719,...,0.0,3.880859,3.830078,0.0,0.290039,2.769531,1.459961,-2.769531,2.179688,-1.209961
258762,44,86,2.48,8.789062,0.0,0.0,0.919922,0.0,0.0,4.511719,...,0.0,3.880859,3.830078,0.0,0.290039,2.769531,1.459961,-2.769531,2.179688,-1.209961


In [20]:
from catboost import CatBoostRanker, Pool, MetricVisualizer, CatBoostRegressor
from copy import deepcopy

In [23]:
train_df.drop(columns=['Rating'])

,UID,JID,j_0,j_1,j_2,j_3,j_4,j_5,j_6,j_7,...,j_90,j_91,j_92,j_93,j_94,j_95,j_96,j_97,j_98,j_99
1393815,3129,51,0.000000,0.000000,0.000000,0.000000,3.109375,7.859375,0.000000,5.441406,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
717533,411,21,-5.628906,-1.309570,0.000000,-6.988281,-1.700195,-2.039062,0.290039,-4.808594,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
740045,21467,60,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.289062,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
258598,18326,16,1.019531,0.000000,-0.150024,-0.189941,0.000000,-1.070312,-4.808594,0.000000,...,3.060547,0.000000,3.449219,1.360352,-3.740234,-5.289062,0.000000,-6.800781,-0.189941,-6.601562
28158,22924,59,-0.970215,0.340088,-3.640625,-5.390625,0.000000,0.000000,-1.459961,0.000000,...,-3.539062,1.070312,2.039062,-3.300781,-1.169922,0.680176,0.239990,0.000000,-0.779785,-2.519531
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193841,24455,41,0.000000,2.230469,-0.290039,1.650391,0.919922,3.500000,0.000000,1.209961,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
768014,864,20,0.000000,0.290039,0.000000,0.000000,3.199219,0.000000,4.761719,4.171875,...,0.000000,0.000000,0.000000,0.000000,1.839844,0.000000,0.000000,0.000000,0.000000,0.000000
661501,21881,62,0.000000,0.000000,-0.290039,7.281250,2.039062,8.828125,-0.870117,-2.039062,...,6.500000,5.730469,8.828125,0.000000,8.828125,2.619141,7.281250,8.250000,0.000000,-0.290039
1192804,24548,4,0.000000,0.000000,0.000000,0.000000,0.000000,3.109375,7.140625,2.330078,...,0.000000,0.000000,0.000000,0.000000,0.000000,-0.919922,0.000000,0.000000,0.000000,0.000000


In [28]:
cat_features = ['UID', 'JID']

In [32]:
train_pool = Pool(train_df.drop(columns=['Rating']), label=train_df['Rating'], cat_features=cat_features)
valid_pool = Pool(valid_df.drop(columns=['Rating']), label=valid_df['Rating'], cat_features=cat_features)
main_pool = Pool(train_joke_df.drop(columns=['Rating']), label=train_joke_df['Rating'], cat_features=cat_features)

test_pool = Pool(test_joke_df_nofactrating, cat_features=cat_features)

In [ ]:
#train = Pool(
#    data=X_train,
#    label=y_train,
#    group_id=queries_train,
#    cat_features=[0, 1]
#)

#test = Pool(
#    data=X_test,
#    label=y_test,
#    group_id=queries_test,
#    cat_features=[0, 1]
#)

In [33]:
default_parameters = {
    'iterations': 3000,
    'custom_metric': 'RMSE',
    'random_seed': 0,
    'train_dir':'RMSE',
    'objective':'RMSE',
    'loss_function':'RMSE',
    'eval_metric':'RMSE',
}


In [34]:
model = CatBoostRegressor(**default_parameters)
model.fit(train_pool, eval_set=valid_pool, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.077231
0:	learn: 5.1492138	test: 5.1458319	best: 5.1458319 (0)	total: 518ms	remaining: 25m 53s
1:	learn: 5.0753908	test: 5.0658587	best: 5.0658587 (1)	total: 806ms	remaining: 20m 8s
2:	learn: 5.0100726	test: 4.9936767	best: 4.9936767 (2)	total: 975ms	remaining: 16m 13s
3:	learn: 4.9527360	test: 4.9295106	best: 4.9295106 (3)	total: 1.14s	remaining: 14m 14s
4:	learn: 4.9009981	test: 4.8738100	best: 4.8738100 (4)	total: 1.32s	remaining: 13m 11s
5:	learn: 4.8554971	test: 4.8239708	best: 4.8239708 (5)	total: 1.51s	remaining: 12m 32s
6:	learn: 4.8163614	test: 4.7799285	best: 4.7799285 (6)	total: 1.8s	remaining: 12m 51s
7:	learn: 4.7803301	test: 4.7409239	best: 4.7409239 (7)	total: 1.97s	remaining: 12m 17s
8:	learn: 4.7481085	test: 4.7061568	best: 4.7061568 (8)	total: 2.16s	remaining: 11m 56s
9:	learn: 4.7186411	test: 4.6757443	best: 4.6757443 (9)	total: 2.52s	remaining: 12m 32s
10:	learn: 4.6934633	test: 4.6483234	best: 4.6483234 (10)	total: 2.85s	remaining: 12m 53s
11

92:	learn: 4.1565254	test: 4.1317975	best: 4.1317975 (92)	total: 37s	remaining: 19m 18s
93:	learn: 4.1515863	test: 4.1270277	best: 4.1270277 (93)	total: 37.4s	remaining: 19m 16s
94:	learn: 4.1487164	test: 4.1244920	best: 4.1244920 (94)	total: 37.8s	remaining: 19m 15s
95:	learn: 4.1453177	test: 4.1212489	best: 4.1212489 (95)	total: 38.2s	remaining: 19m 14s
96:	learn: 4.1434769	test: 4.1194538	best: 4.1194538 (96)	total: 38.5s	remaining: 19m 12s
97:	learn: 4.1406947	test: 4.1169998	best: 4.1169998 (97)	total: 38.9s	remaining: 19m 11s
98:	learn: 4.1366455	test: 4.1132964	best: 4.1132964 (98)	total: 39.4s	remaining: 19m 13s
99:	learn: 4.1322938	test: 4.1090859	best: 4.1090859 (99)	total: 40s	remaining: 19m 19s
100:	learn: 4.1301073	test: 4.1070331	best: 4.1070331 (100)	total: 40.3s	remaining: 19m 17s
101:	learn: 4.1276823	test: 4.1049391	best: 4.1049391 (101)	total: 40.8s	remaining: 19m 20s
102:	learn: 4.1257835	test: 4.1033186	best: 4.1033186 (102)	total: 41.3s	remaining: 19m 21s
103:	lea

182:	learn: 3.9317437	test: 3.9209419	best: 3.9209419 (182)	total: 1m 15s	remaining: 19m 20s
183:	learn: 3.9293084	test: 3.9185936	best: 3.9185936 (183)	total: 1m 15s	remaining: 19m 20s
184:	learn: 3.9269386	test: 3.9162040	best: 3.9162040 (184)	total: 1m 16s	remaining: 19m 21s
185:	learn: 3.9236847	test: 3.9130612	best: 3.9130612 (185)	total: 1m 16s	remaining: 19m 22s
186:	learn: 3.9218324	test: 3.9112900	best: 3.9112900 (186)	total: 1m 17s	remaining: 19m 20s
187:	learn: 3.9204617	test: 3.9100546	best: 3.9100546 (187)	total: 1m 17s	remaining: 19m 18s
188:	learn: 3.9190911	test: 3.9087365	best: 3.9087365 (188)	total: 1m 17s	remaining: 19m 17s
189:	learn: 3.9175303	test: 3.9072796	best: 3.9072796 (189)	total: 1m 18s	remaining: 19m 15s
190:	learn: 3.9161588	test: 3.9060451	best: 3.9060451 (190)	total: 1m 18s	remaining: 19m 14s
191:	learn: 3.9149928	test: 3.9050057	best: 3.9050057 (191)	total: 1m 18s	remaining: 19m 15s
192:	learn: 3.9112074	test: 3.9013411	best: 3.9013411 (192)	total: 1m 

271:	learn: 3.7604524	test: 3.7572130	best: 3.7572130 (271)	total: 1m 49s	remaining: 18m 16s
272:	learn: 3.7593343	test: 3.7560870	best: 3.7560870 (272)	total: 1m 49s	remaining: 18m 16s
273:	learn: 3.7580664	test: 3.7548655	best: 3.7548655 (273)	total: 1m 50s	remaining: 18m 17s
274:	learn: 3.7549892	test: 3.7518245	best: 3.7518245 (274)	total: 1m 50s	remaining: 18m 18s
275:	learn: 3.7532806	test: 3.7502510	best: 3.7502510 (275)	total: 1m 51s	remaining: 18m 16s
276:	learn: 3.7516673	test: 3.7487688	best: 3.7487688 (276)	total: 1m 51s	remaining: 18m 16s
277:	learn: 3.7496449	test: 3.7468198	best: 3.7468198 (277)	total: 1m 51s	remaining: 18m 15s
278:	learn: 3.7490543	test: 3.7463058	best: 3.7463058 (278)	total: 1m 52s	remaining: 18m 13s
279:	learn: 3.7478563	test: 3.7452066	best: 3.7452066 (279)	total: 1m 52s	remaining: 18m 12s
280:	learn: 3.7464979	test: 3.7439217	best: 3.7439217 (280)	total: 1m 52s	remaining: 18m 11s
281:	learn: 3.7433323	test: 3.7409977	best: 3.7409977 (281)	total: 1m 

...

2636:	learn: 1.9778791	test: 1.9878770	best: 1.9878770 (2636)	total: 18m 26s	remaining: 2m 32s
2637:	learn: 1.9766640	test: 1.9868121	best: 1.9868121 (2637)	total: 18m 26s	remaining: 2m 31s
2638:	learn: 1.9762378	test: 1.9863717	best: 1.9863717 (2638)	total: 18m 27s	remaining: 2m 31s
2639:	learn: 1.9758176	test: 1.9859847	best: 1.9859847 (2639)	total: 18m 27s	remaining: 2m 31s
2640:	learn: 1.9756998	test: 1.9858808	best: 1.9858808 (2640)	total: 18m 27s	remaining: 2m 30s
2641:	learn: 1.9755572	test: 1.9857544	best: 1.9857544 (2641)	total: 18m 28s	remaining: 2m 30s
2642:	learn: 1.9750804	test: 1.9852928	best: 1.9852928 (2642)	total: 18m 29s	remaining: 2m 29s
2643:	learn: 1.9743098	test: 1.9845541	best: 1.9845541 (2643)	total: 18m 29s	remaining: 2m 29s
2644:	learn: 1.9740286	test: 1.9842869	best: 1.9842869 (2644)	total: 18m 29s	remaining: 2m 28s
2645:	learn: 1.9737457	test: 1.9840210	best: 1.9840210 (2645)	total: 18m 30s	remaining: 2m 28s
2646:	learn: 1.9736492	test: 1.9839397	best: 1.983

2723:	learn: 1.9512441	test: 1.9615261	best: 1.9615261 (2723)	total: 19m 6s	remaining: 1m 56s
2724:	learn: 1.9508236	test: 1.9610838	best: 1.9610838 (2724)	total: 19m 7s	remaining: 1m 55s
2725:	learn: 1.9505700	test: 1.9608310	best: 1.9608310 (2725)	total: 19m 7s	remaining: 1m 55s
2726:	learn: 1.9501287	test: 1.9603923	best: 1.9603923 (2726)	total: 19m 8s	remaining: 1m 54s
2727:	learn: 1.9494630	test: 1.9596859	best: 1.9596859 (2727)	total: 19m 8s	remaining: 1m 54s
2728:	learn: 1.9490416	test: 1.9592454	best: 1.9592454 (2728)	total: 19m 9s	remaining: 1m 54s
2729:	learn: 1.9487794	test: 1.9589806	best: 1.9589806 (2729)	total: 19m 9s	remaining: 1m 53s
2730:	learn: 1.9485181	test: 1.9587364	best: 1.9587364 (2730)	total: 19m 9s	remaining: 1m 53s
2731:	learn: 1.9483264	test: 1.9585420	best: 1.9585420 (2731)	total: 19m 10s	remaining: 1m 52s
2732:	learn: 1.9482210	test: 1.9584144	best: 1.9584144 (2732)	total: 19m 10s	remaining: 1m 52s
2733:	learn: 1.9476093	test: 1.9578465	best: 1.9578465 (27

2810:	learn: 1.9196012	test: 1.9301578	best: 1.9301578 (2810)	total: 19m 46s	remaining: 1m 19s
2811:	learn: 1.9194011	test: 1.9299375	best: 1.9299375 (2811)	total: 19m 47s	remaining: 1m 19s
2812:	learn: 1.9185763	test: 1.9292410	best: 1.9292410 (2812)	total: 19m 47s	remaining: 1m 18s
2813:	learn: 1.9178844	test: 1.9285749	best: 1.9285749 (2813)	total: 19m 47s	remaining: 1m 18s
2814:	learn: 1.9170372	test: 1.9277240	best: 1.9277240 (2814)	total: 19m 48s	remaining: 1m 18s
2815:	learn: 1.9167985	test: 1.9274793	best: 1.9274793 (2815)	total: 19m 49s	remaining: 1m 17s
2816:	learn: 1.9161766	test: 1.9268741	best: 1.9268741 (2816)	total: 19m 49s	remaining: 1m 17s
2817:	learn: 1.9159617	test: 1.9266629	best: 1.9266629 (2817)	total: 19m 49s	remaining: 1m 16s
2818:	learn: 1.9158235	test: 1.9265294	best: 1.9265294 (2818)	total: 19m 50s	remaining: 1m 16s
2819:	learn: 1.9155539	test: 1.9262784	best: 1.9262784 (2819)	total: 19m 51s	remaining: 1m 16s
2820:	learn: 1.9149918	test: 1.9256830	best: 1.925

2898:	learn: 1.8843936	test: 1.8949053	best: 1.8949053 (2898)	total: 20m 29s	remaining: 42.8s
2899:	learn: 1.8837457	test: 1.8942758	best: 1.8942758 (2899)	total: 20m 30s	remaining: 42.4s
2900:	learn: 1.8836760	test: 1.8942206	best: 1.8942206 (2900)	total: 20m 30s	remaining: 42s
2901:	learn: 1.8835687	test: 1.8941050	best: 1.8941050 (2901)	total: 20m 31s	remaining: 41.6s
2902:	learn: 1.8832927	test: 1.8938191	best: 1.8938191 (2902)	total: 20m 31s	remaining: 41.2s
2903:	learn: 1.8829081	test: 1.8934706	best: 1.8934706 (2903)	total: 20m 32s	remaining: 40.7s
2904:	learn: 1.8826287	test: 1.8931889	best: 1.8931889 (2904)	total: 20m 32s	remaining: 40.3s
2905:	learn: 1.8825364	test: 1.8930975	best: 1.8930975 (2905)	total: 20m 33s	remaining: 39.9s
2906:	learn: 1.8822527	test: 1.8928094	best: 1.8928094 (2906)	total: 20m 33s	remaining: 39.5s
2907:	learn: 1.8816033	test: 1.8921215	best: 1.8921215 (2907)	total: 20m 34s	remaining: 39s
2908:	learn: 1.8815530	test: 1.8920694	best: 1.8920694 (2908)	to

2986:	learn: 1.8481852	test: 1.8588856	best: 1.8588856 (2986)	total: 21m 11s	remaining: 5.53s
2987:	learn: 1.8470849	test: 1.8577759	best: 1.8577759 (2987)	total: 21m 11s	remaining: 5.11s
2988:	learn: 1.8463366	test: 1.8570567	best: 1.8570567 (2988)	total: 21m 12s	remaining: 4.68s
2989:	learn: 1.8455202	test: 1.8562212	best: 1.8562212 (2989)	total: 21m 13s	remaining: 4.26s
2990:	learn: 1.8449184	test: 1.8556416	best: 1.8556416 (2990)	total: 21m 13s	remaining: 3.83s
2991:	learn: 1.8444102	test: 1.8551077	best: 1.8551077 (2991)	total: 21m 13s	remaining: 3.41s
2992:	learn: 1.8441051	test: 1.8547844	best: 1.8547844 (2992)	total: 21m 14s	remaining: 2.98s
2993:	learn: 1.8432983	test: 1.8539323	best: 1.8539323 (2993)	total: 21m 15s	remaining: 2.56s
2994:	learn: 1.8425526	test: 1.8532076	best: 1.8532076 (2994)	total: 21m 15s	remaining: 2.13s
2995:	learn: 1.8418585	test: 1.8525486	best: 1.8525486 (2995)	total: 21m 15s	remaining: 1.7s
2996:	learn: 1.8414532	test: 1.8521353	best: 1.8521353 (2996)

In [ ]:
assert False

In [ ]:
predict = model.predict(valid_pool)
print(mean_squared_error(valid_df['Rating'].values, predict, squared=False))


In [ ]:
predict = model.predict(test_pool)

test_joke_df_nofactrating['Rating'] = predict

display(test_joke_df_nofactrating['Rating'].to_frame().head(5))
test_joke_df_nofactrating['Rating'].to_frame().to_csv('catboost0.9.csv')

In [ ]:
parameters = {
    'iterations': 2000,
    'custom_metric': 'RMSE',
    'random_seed': 0,
    'train_dir':'RMSE',
    'objective':'RMSE',
    'loss_function':'RMSE',
    'eval_metric':'RMSE',
}

model = CatBoostRanker(**parameters)
model.fit(main_pool, eval_set=valid_pool, plot=True)

In [ ]:
predict = model.predict(valid_pool)
print(mean_squared_error(valid_df['Rating'].values, predict, squared=False))


In [ ]:
predict = model.predict(test_pool)

test_joke_df_nofactrating['Rating'] = predict

display(test_joke_df_nofactrating['Rating'].to_frame().head(5))
test_joke_df_nofactrating['Rating'].to_frame().to_csv('catboost.csv')